## 네이버 뉴스 수집기
---
### 사용한 버젼

In [178]:
# %load_ext watermark
%watermark -adatacampus-ku -d -v --iversions

Author: datacampus-ku

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.20.0

json   : 2.0.9
urllib3: 1.26.3
parser : 0.5
re     : 2.2.1



In [177]:
import json
import re
import urllib.request
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from requests.sessions import Session
from time import sleep

In [6]:
def download(url, params={}, headers={}, method='GET', maximum=2):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [172]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

In [174]:
def download_video(url):
    save_file = './MOV/temp.mp4'
    urllib.request.urlretrieve(url, save_file)
    
    return True

In [24]:
# options = ChromeOptions()
# options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36")
# driver = Chrome(options=options)

headers = {
    'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'
}

In [179]:
# 찾을 뉴스 URL
url = "https://news.naver.com/main/tvgallery/replay.naver?sid=751&date=20200101#052_0001383422"

# 동영상을 다운받을 서버
base_url = "https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0"

In [165]:
# Network에서 XHR로 통신하는 링크 : 
request_url = "https://apis.naver.com/rmcnmv/rmcnmv/vod/play/v2.0/BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46?key=V125b02e82c56f176158790ee53ef26e33e09d79e50772ed27e1b5d2aa1623c99302a90ee53ef26e33e09&pid=rmcPlayer_16269668385546947&sid=2006&ver=2.0&devt=html5_mo&doct=json&ptc=https&sptc=https&cpt=vtt&ctls=%7B%22visible%22%3A%7B%22fullscreen%22%3Atrue%2C%22logo%22%3Afalse%2C%22playbackRate%22%3Afalse%2C%22scrap%22%3Afalse%2C%22playCount%22%3Atrue%2C%22commentCount%22%3Atrue%2C%22title%22%3Atrue%2C%22writer%22%3Atrue%2C%22expand%22%3Afalse%2C%22subtitles%22%3Atrue%2C%22thumbnails%22%3Atrue%2C%22quality%22%3Atrue%2C%22setting%22%3Atrue%2C%22script%22%3Afalse%2C%22logoDimmed%22%3Atrue%2C%22badge%22%3Atrue%2C%22seekingTime%22%3Atrue%2C%22muted%22%3Atrue%2C%22muteButton%22%3Afalse%2C%22viewerNotice%22%3Afalse%2C%22linkCount%22%3Afalse%2C%22createTime%22%3Afalse%2C%22thumbnail%22%3Atrue%7D%2C%22clicked%22%3A%7B%22expand%22%3Afalse%2C%22subtitles%22%3Afalse%7D%7D&pv=4.18.40&dr=2560x1440&lc=ko_KR&videoId=BAA1BD447C6AEAAAFC86174FC5C8BA8DCF46"

In [70]:
# data에 parameter들을 나눠서 저장
keys = re.findall(r'\?(key=.+)', request_url)[0]
data = parse_qs(urljoin(base=base_url, url=''.join(['&', keys])))

In [74]:
# 동영상 링크 주소를 얻기 위해 GET
respond = download(''.join([temp, '/', data['videoId'][0], '?', keys]), headers=headers)

In [141]:
rmcPlayer = json.dumps(respond.json()['videos'])

In [163]:
# 최종 뉴스 720P 영상 링크
video_link = re.findall(r'"720P".+?"source": "(.+)",', rmcPlayer)[0]

In [175]:
# 다운로드 속도가 생각보다 많이 느리다.. 31MB를 다운받는데 20초 걸린듯?
download_video(video_link)

True